In [403]:
import pandas as pd
import csv
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn.utils import shuffle
import os

In [400]:
eng_freq_fn = '/mnt/permanent/home/eszti/dipterv/panlex/data/freq/eng_vocab_freq.txt'
ita_freq_fn = '/mnt/permanent/home/eszti/dipterv/panlex/data/freq/ita_vocab_freq_no_space.txt'

conf = 7

pl_fn = '/mnt/permanent/home/eszti/dipterv/panlex/data/panlex/tsv/eng_ita_tr_te/{0}/eng_ita.tsv'.format(conf)
train_fn = '/mnt/permanent/home/eszti/dipterv/panlex/data/panlex/tsv/freq/{0}/train/eng_ita.tsv'.format(conf)
test_fn = '/mnt/permanent/home/eszti/dipterv/panlex/data/panlex/tsv/freq/{0}/test/eng_ita.tsv'.format(conf)

In [263]:
# Read eng freq info
eng_f = pd.read_csv(eng_freq_fn, sep=' ', header=None, names = ["eng", "freq"])
eng_f.freq = eng_f.freq.astype(int)
eng_f.head()
len(eng_f.index)

,eng,freq
0,the,193688441
1,",",152169667
2,.,130021704
3,of,101645398
4,and,94288110


6620170

In [264]:
# Read ita freq info 
ita_ls = []
with open(ita_freq_fn, encoding='latin-1') as f:
    lines = f.read().splitlines()
    for i, line in enumerate(lines):
        words = line.split(' ')
        try:
            ita_ls.append((words[0], words[1]))
        except:
            continue
len(ita_ls)

ita_f = pd.DataFrame(ita_ls, columns=['freq', 'ita'])
ita_f.freq = ita_f.freq.astype(int)
len(ita_f.index)
ita_f.head()

6278167

6278167

,freq,ita
0,98847888,","
1,65521116,di
2,56115098,.
3,40599023,e
4,27182874,che


In [356]:
# Read panlex data
pl = pd.read_csv(pl_fn, sep='\t', header=None, names = ["eng", "ita"])
pl.head()
len(pl.index)

,eng,ita
0,dirham,dirham
1,dinar,dinaro
2,dong,dong
3,lari,lari
4,dobra,dobra


69623

In [357]:
# Merge eng freq info
pl_f = pd.merge(pl, eng_f)
len(pl_f.index)

69567

In [358]:
# Merge ita freq info
pl_ff = pd.merge(pl_f, ita_f, on=['ita'])
len(pl_ff.index)

69504

In [359]:
# Create bins from English

bin1 = eng_f.iloc[1000:5000]
bin2 = eng_f.iloc[5000:20000]
bin3 = eng_f.iloc[20000:50000]
bin4 = eng_f.iloc[50000:100000]
bin5 = eng_f.iloc[100000:200000]

In [360]:
def get_300_uq(binN, alle, N, uq=True):
    print('Number of words in {0}: {1}'.format(N, len(binN.index)))
    
    alle_bin = pd.merge(alle, binN, on=['eng'])
    print('len alle_bin: {0}'.format(len(alle_bin.index)))
    
    eng_bin = alle_bin[['eng']].copy().drop_duplicates()
    print('len eng_bin: {0}'.format(len(eng_bin.index)))
    
    eng_shuf = shuffle(eng_bin)
    
    eng_300 = eng_shuf.head(300)
    print('len eng_300: {0}'.format(len(eng_300.index)))
    
    testN = pd.merge(alle, eng_300, on=['eng'])
    print('lenn testN: {0}'.format(len(testN.index)))
    
    testN_uq = None    
    if uq:
        testN_uq = testN.sort_values('freq_y', ascending=False)
        testN_uq = testN_uq.drop_duplicates(subset=['eng'])
        print('len testN uq: {0}'.format(len(testN_uq.index)))

    return testN, testN_uq

In [361]:
test1, test1_uq = get_300_uq(bin1, pl_ff, 1)

Number of words in 1: 4000
len alle_bin: 14306
len eng_bin: 2719
len eng_300: 300
lenn testN: 1589
len testN uq: 300


In [362]:
test2, test2_uq = get_300_uq(bin2, pl_ff, 2)

Number of words in 2: 15000
len alle_bin: 23115
len eng_bin: 7465
len eng_300: 300
lenn testN: 885
len testN uq: 300


In [363]:
test3, test3_uq = get_300_uq(bin3, pl_ff, 3)

Number of words in 3: 30000
len alle_bin: 15754
len eng_bin: 7845
len eng_300: 300
lenn testN: 583
len testN uq: 300


In [364]:
test4, test4_uq = get_300_uq(bin4, pl_ff, 4)

Number of words in 4: 50000
len alle_bin: 6805
len eng_bin: 4418
len eng_300: 300
lenn testN: 455
len testN uq: 300


In [365]:
test5, test5_uq = get_300_uq(bin5, pl_ff, 5)

Number of words in 5: 100000
len alle_bin: 2202
len eng_bin: 1700
len eng_300: 300
lenn testN: 384
len testN uq: 300


In [374]:
# Concat test

frames = [test1_uq, test2_uq, test3_uq, test4_uq, test5_uq]
test = pd.concat(frames)
len(test.index)

1500

In [419]:
# Create train with multiple meanings

test_eng = test[['eng']].copy()
len(test_eng.index)

pl_train = pl_ff[~pl_ff.eng.isin(set(test_eng.eng))]
len(pl_ff.index)
len(pl_train.index)

pl_train = pl_train.sort_values(['freq_x', 'freq_y'], ascending=False)
pl_train_5K = pl_train.iloc[0:5000]
pl_train_5K.iloc[980:1010]
len(pl_train_5K['eng'].drop_duplicates())

# Create train without multiple meanings

pl_train_uq = pl_train.drop_duplicates(subset=['eng'])
pl_train_5K_uq = pl_train_uq.iloc[0:5000]
pl_train_5K_uq.head(10)
len(pl_train_5K_uq['eng'].drop_duplicates())

1500

69504

65608

,eng,ita,freq_x,freq_y
5131,number,valutare,1951249,91838
5114,number,cifra,1951249,46596
5123,number,ammontare,1951249,46018
5126,number,quantitativo,1951249,16345
5152,number,stimare,1951249,4420
5106,number,numerare,1951249,368
5121,number,numerale,1951249,203
1898,see,vedere,1948604,345209
1923,see,capire,1948604,212892
1908,see,leggere,1948604,143518


619

,eng,ita,freq_x,freq_y
43599,the,la,193688441,25477273
10400,",",",",152169667,98847888
10399,.,",",130021704,98847888
4929,of,di,101645398,65521116
69243,and,e,94288110,40599023
6507,to,in,87022181,21843465
6466,a,a,62172985,19971959
6505,in,in,60930962,21843465
43654,that,che,36294963,27182874
65805,is,è,34782166,14929934


5000

In [399]:
test[test.eng == 'cat']

pl_train_5K_uq[pl_train_5K_uq.eng == 'cat']

pl_train[pl_train.eng == 'cat']

,eng,ita,freq_x,freq_y


,eng,ita,freq_x,freq_y
11668,cat,gatto,47148,22226


,eng,ita,freq_x,freq_y
11668,cat,gatto,47148,22226
11671,cat,gatta,47148,3726
11680,cat,felino,47148,1709
11673,cat,micio,47148,1197
11679,cat,felina,47148,882


In [410]:
eng_te = set(test['eng'])
eng_tr = set(pl_train_5K_uq['eng'])
len(eng_te & eng_tr)

ita_te = set(test['ita'])
ita_tr = set(pl_train_5K_uq['ita'])
len(ita_te & ita_tr)

0

298

In [407]:
def save_tsv(df, fn):
    if not os.path.exists(os.path.dirname(fn)):
        os.makedirs(os.path.dirname(fn))
    pd.DataFrame.to_csv(df, columns=['eng', 'ita'], header=False, path_or_buf=fn, sep='\t', index=False)

In [408]:
save_tsv(pl_train_5K_uq, train_fn)
save_tsv(test, test_fn)